In [ ]:
# ###################################
#     ----    S T A R T ----
# ###################################

# Libraries to be installed every run

!pip install moviepy;
!pip install pytube ;
!apt install imagemagick;


# ###################################
#       -- I M P O R T A N T --
# ###################################
# imagemagick have a piece of line that needs deleting so this would read and delete the line ( rewrite file )

policypath = '/etc/ImageMagick-6/policy.xml'
try:
  with open(policypath,'r') as f:
    lines = f.readlines()
  # del policypath
except:
  print('File not found...')

# policypath = '/etc/ImageMagick-6/policy.xml'
with open(policypath,'w') as rwrite:
  for i in lines:
    if i.strip() == '<policy domain="path" rights="none" pattern="@*"/>':
      print('deleted')
    else:
      rwrite.write(i)

In [2]:
import sys
import os
from pytube import YouTube # downloading videos
from moviepy.editor import * # for editing video
import subprocess # splitting videos
import re

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2326528/45929032 bytes (5.1%)5734400/45929032 bytes (12.5%)9043968/45929032 bytes (19.7%)12566528/45929032 bytes (27.4%)16130048/45929032 bytes (35.1%)19750912/45929032 bytes (43.0%)23298048/45929032 bytes (50.7%)26828800/45929032 bytes (58.4%)30334976/45929032 bytes (66.0%)34029568/45929032 bytes (74.1%)37584896/45929032 bytes (81.8%)41156608/45929032 bytes (89.6%)

In [7]:
def download(Link):
    cwdpath = os.getcwd()
    print(f' Your location : {cwdpath}')

    # DOWNLOAD YOUTUBE VIDEOS
    print(' Let\'s download a Video ')
    yt = YouTube(Link)
    title = yt.title
    print(f' Video Name      : {title}')
    desc = yt.description
    lenght = yt.length
    print(f' Length of video : {lenght} seconds')

    video = yt.streams.get_highest_resolution()
    vid_loc = os.path.join(cwdpath,'contents')
    path = video.download(vid_loc)

    document = {
        'Title'          : title,
        'Description'    : desc,
        'Video_Path'     : path,
        'Current_Folder' : cwdpath
    }
    with open('/content/info.txt','w') as f:
      f.write(title+'\n'+desc+'\n')

    return document

def manifest(start,end):
  path = '/content/contents/Nvideo.mp4'
  with open('/content/manifest.csv','w') as f:
    f.write('start_time,length,rename_to\n')
    f.write(f'{start},{end},{path}')
  print('WRITING TO MANIFEST FILE')

def split(meta,split_time,start,end):
  #SPLIT VIDEO
  path  = meta['Video_Path'] 

  splitter = '/content/video-splitter/ffmpeg-split.py'
  if os.path.isfile(splitter) :
    print('File is available. Moving along.')
  else:
    !git clone https://github.com/c0decracker/video-splitter.git
  if end == 0:
    subprocess.run(['python', splitter, '-s',split_time ,'-f', path ])
  else:
    manifest(start,end)
    subprocess.run(['python', splitter ,'-f', path, '-m','/content/manifest.csv' ])
    os. remove(path)

    path = '/content/contents/Nvideo.mp4'

    subprocess.run(['python', splitter, '-s', split_time  ,'-f', path])

  os. remove(path) 


def changeVideo():
  lists = os.listdir('/content/contents/')
  x= 0
  paths = []
  for i in lists:
    paths.append(os.path.join('/content/contents',i))
    print(paths[x])
    x+=1
    regex = '(-\d-\d*)'
  info = '/content/info.txt'
  with open(info,'r') as f:
    p = f.readline()
  Name = p.replace('(Official Video)\n','').strip()
  path = paths[0]

  total = len(paths)
  num = 1
  for path in paths:
    part = re.findall('-(\w+)-',path)[0]
    try:
      print( '---------------------------------------------')
      print(f'Video Part : {num} / {total}')
      print( '---------------------------------------------')
      videoE = VideoFileClip(path, audio=True)
      w,h = moviesize = videoE.size

      txt = TextClip(f' {Name}\n Part : {part} - {total}', font='Amiri-regular',
                      color='yellow',fontsize=30);
      txt = txt.set_position(lambda t:(10,50)).set_duration(videoE.duration);
      video = CompositeVideoClip([videoE,txt.set_start(0.1).crossfadein(2)]);
      # txt = TextClip(f'Part : {part} - {total}', font='Amiri-regular',
      #                 color='yellow',fontsize=20);
      # txt = txt.set_position(lambda t:(20,80)).set_duration(video.duration);
      # video = CompositeVideoClip([video,txt.set_start(0.1).crossfadein(2)]);
      name = Name.replace(' ','')
      video.write_videofile(f'/content/contents/{name}_{part}.mp4');
      #clean up 
      os.remove(path)
    except:
      print(f'Something Wrong With Part : {part}')
    num+=1
def videoSavior():
  #Zip Folder and Download
  from google.colab import files
  !zip -r /content/contents/Videos.zip /content/contents/
  files.download("/content/contents/Videos.zip")

def cleanUP():
  dir = os.listdir('/content/contents/')
  print('Cleaning up Storage...')
  for i in dir:
    os.remove('/content/contents/'+i)
  print('Still Cleaning...')
  os.remove('/content/info.txt')
  os.remove('/content/manifest.csv')

def main(split_time,start,end):
    Link = input(' Please Enter YouTube Link : ')

    meta = download(Link)

    split(meta,str(split_time),start,end)

    changeVideo()

    videoSavior()
    
    cleanUP()

In [ ]:
# --------------------------------
#     --  Start Program   --

# Change these 2 variables only if you're splitting main video before splitting to 59s videos
start      = 5  # 0 default
end        = 299  # 0 default
split_time = 59 # time for videos. 59 secs for tiktoks

main(split_time,start,end) 

## Enter Desired Video to Upload to Status / Tiktok

In [ ]:
https://www.youtube.com/watch?v=9EtKgmVMsFQ&ab_channel=Psych2Go

In [6]:
dir = os.listdir('/content/contents/')
for i in dir:
  os.remove('/content/contents/'+i)
os.remove